In [27]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings  # updated import
from langchain_community.vectorstores import FAISS

In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import pandas as pd

books = pd.read_csv(r"C:\Users\MSI\Desktop\Book Sphere\data\books_cleaned.csv")
print(books.head())


          isbn13      isbn10                title  \
0  9780002005883  0002005883               Gilead   
1  9780002261982  0002261987         Spider's Web   
2  9780006178736  0006178731       Rage of angels   
3  9780006280897  0006280897       The Four Loves   
4  9780006280934  0006280935  The Problem of Pain   

                           authors                     categories  \
0               Marilynne Robinson                        Fiction   
1  Charles Osborne;Agatha Christie  Detective and mystery stories   
2                   Sidney Sheldon                        Fiction   
3              Clive Staples Lewis                 Christian life   
4              Clive Staples Lewis                 Christian life   

                                           thumbnail  \
0  http://books.google.com/books/content?id=KQZCP...   
1  http://books.google.com/books/content?id=gA5GP...   
2  http://books.google.com/books/content?id=FKo2T...   
3  http://books.google.com/books/content?i

In [7]:
with open("../tagged_description.txt", "w", encoding="utf-8") as f:
    for line in books["tagged_description"].tolist():
        f.write(line + "\n")


In [12]:
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document

# Load document
raw_documents = TextLoader("../tagged_description.txt", encoding="utf-8").load()

# Manually split by newline (like chunk_size=0)
documents = []
for doc in raw_documents:
    lines = doc.page_content.split("\n")
    for line in lines:
        if line.strip():  # skip empty lines
            documents.append(Document(page_content=line))

print(f"Total chunks created: {len(documents)}")


Total chunks created: 5197


In [13]:
documents[0]

Document(metadata={}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and 

In [36]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Example usage with Chroma
texts = ["This is a book about AI.", "This is a novel about love."]
db = Chroma.from_texts(texts, embeddings, persist_directory="db_books")

# Test search
query = "romantic stories"
results = db.similarity_search(query, k=1)
print(results)



C:\Users\MSI\Desktop\Book Sphere\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={}, page_content='This is a novel about love.')]


In [37]:
!pip install -U ipywidgets


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)

   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]



In [39]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Example usage with Chroma
texts = ["This is a book about AI.", "This is a novel about love."]
db = Chroma.from_texts(texts, embeddings, persist_directory="db_books")

# Test search
query = "romantic stories"
results = db.similarity_search(query, k=10)
print(results)



[Document(metadata={}, page_content='This is a novel about love.'), Document(metadata={}, page_content='This is a novel about love.'), Document(metadata={}, page_content='This is a novel about love.'), Document(metadata={}, page_content='This is a novel about love.'), Document(metadata={}, page_content='This is a novel about love.'), Document(metadata={}, page_content='This is a novel about love.'), Document(metadata={}, page_content='This is a book about AI.'), Document(metadata={}, page_content='This is a book about AI.'), Document(metadata={}, page_content='This is a book about AI.'), Document(metadata={}, page_content='This is a book about AI.')]


In [42]:
query = "romantic stories"
results = db.similarity_search(query, k=1)  # or k=10

# The first document
first_doc = results[0]

print(first_doc.page_content)


This is a novel about love.


In [43]:
isbn_str = first_doc.page_content.split("\n")[0].strip()


In [44]:
try:
    isbn_num = int(isbn_str)
except ValueError:
    print(f"Cannot convert ISBN to int: {isbn_str}")


Cannot convert ISBN to int: This is a novel about love.


In [48]:
from langchain.schema import Document

# Create documents with ISBNs in metadata
documents = []
for idx, row in books.iterrows():
    documents.append(
        Document(
            page_content=row['tagged_description'],
            metadata={"isbn13": row['isbn13']}
        )
    )

# Build Chroma DB from these documents
db = Chroma.from_documents(documents, embeddings, persist_directory="db_books")

# Example search
query = "romantic stories"
results = db.similarity_search(query, k=1)

# Get ISBN from metadata
first_doc = results[0]
isbn_num = first_doc.metadata["isbn13"]
print("Matched ISBN:", isbn_num)

# Get book info from DataFrame
matched_books = books[books["isbn13"] == isbn_num]
print(matched_books)


Matched ISBN: 9780141187198
            isbn13      isbn10               title       authors categories  \
768  9780141187198  0141187190  Seven Gothic Tales  Isak Dinesen    Fiction   

                                             thumbnail  \
768  http://books.google.com/books/content?id=4uKAn...   

                                           description  published_year  \
768  A selection of stories about romantics, advent...          2002.0   

     average_rating  num_pages  ratings_count  title_and_subtitle  \
768            3.94      368.0           82.0  Seven Gothic Tales   

                                    tagged_description  
768  9780141187198 A selection of stories about rom...  


In [49]:
# Example: search query
query = "books for babies about animals"
docs = db.similarity_search(query, k=10)


In [50]:
# Suppose the ISBN is the first number in the document content
doc_content = docs[0].page_content.strip()

# Remove extra spaces and split if needed
isbn_num = ''.join(filter(str.isdigit, doc_content))  # keep only digits
isbn_num = int(isbn_num)


In [51]:
matched_books = books[books["isbn13"] == isbn_num]

print(matched_books)


             isbn13      isbn10                  title             authors  \
3749  9780786808380  0786808381  Baby Einstein: Babies  Julie Aigner-Clark   

            categories                                          thumbnail  \
3749  Juvenile Fiction  http://books.google.com/books/content?id=jv4NA...   

                                            description  published_year  \
3749  Introduce your babies to birds, cats, dogs, an...          2002.0   

      average_rating  num_pages  ratings_count     title_and_subtitle  \
3749            4.03       20.0           29.0  Baby Einstein: Babies   

                                     tagged_description  
3749  9780786808380 Introduce your babies to birds, ...  


In [52]:
doc_content = docs[0].page_content.strip()
isbn_str = doc_content.split()[0]  # take first word if ISBN is at start
isbn_num = int(''.join(filter(str.isdigit, isbn_str)))  # keep only digits


In [53]:
matched_books = books[books["isbn13"] == isbn_num]
print(matched_books)


             isbn13      isbn10                  title             authors  \
3749  9780786808380  0786808381  Baby Einstein: Babies  Julie Aigner-Clark   

            categories                                          thumbnail  \
3749  Juvenile Fiction  http://books.google.com/books/content?id=jv4NA...   

                                            description  published_year  \
3749  Introduce your babies to birds, cats, dogs, an...          2002.0   

      average_rating  num_pages  ratings_count     title_and_subtitle  \
3749            4.03       20.0           29.0  Baby Einstein: Babies   

                                     tagged_description  
3749  9780786808380 Introduce your babies to birds, ...  


In [57]:
# Function to retrieve semantic recommendations
def retrieve_semantic_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    # Step 1: Semantic search in Chroma DB
    recs = db_books.similarity_search(query, k=50)  # search top 50 similar docs

    # Step 2: Extract ISBNs from the search results
    books_list = []
    for doc in recs[8:]:  # skip first 8 if needed
        try:
            isbn_str = doc.page_content.strip().split()[0]
            isbn_num = int(''.join(filter(str.isdigit, isbn_str)))
            books_list.append(isbn_num)
        except:
            continue  # skip if conversion fails

    # Step 3: Return matching books from CSV
    return books[books["isbn13"].isin(books_list)].head(top_k)


In [58]:
recommendations = retrieve_semantic_recommendations("A book to teach children about animals")
print(recommendations)


NameError: name 'db_books' is not defined

In [59]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Assuming you already have your documents split into 'documents'
# or you can use your texts column from the CSV
texts = books["tagged_description"].tolist()

# Build/load Chroma DB
db_books = Chroma.from_texts(texts, embeddings, persist_directory="db_books")

# Optional: persist the database (Chroma auto-persists in newer versions)
db_books.persist()


C:\Users\MSI\AppData\Local\Temp\ipykernel_27160\1824184229.py:15: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db_books.persist()


In [60]:
recommendations = retrieve_semantic_recommendations("A book to teach children about animals")
print(recommendations)


             isbn13      isbn10                                title  \
368   9780061127762  0061127760    Charlotte's Web Signature Edition   
404   9780064402453  0064402452           Racso and the Rats of NIMH   
406   9780064403870  0064403874  R-T, Margaret, and the Rats of NIMH   
407   9780064404419  0064404412                   The Rainbow People   
416   9780064406925  006440692X                   Winter on the Farm   
429   9780064434980  0064434982                 The Deer in the Wood   
812   9780142302279  0142302279                         Dirty Beasts   
1078  9780241003008  0241003008          The Very Hungry Caterpillar   
1222  9780312319731  0312319738                             Dominion   
1479  9780345434906  0345434900               The Real James Herriot   

                    authors                 categories  \
368             E. B. White           Juvenile Fiction   
404       Jane Leslie Conly           Juvenile Fiction   
406       Jane Leslie Conly      

In [61]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


def retrieve_semantic_recommendations(query: str, db: Chroma, books_df: pd.DataFrame, top_k: int = 10) -> pd.DataFrame:
    """
    Perform semantic search on a book vector database and return top matching books.

    Parameters:
        query (str): User query string.
        db (Chroma): Chroma vector database containing book embeddings.
        books_df (pd.DataFrame): Original book dataset with ISBNs and metadata.
        top_k (int): Number of recommendations to return.

    Returns:
        pd.DataFrame: Top-k recommended books with full metadata.
    """
    # Step 1: Semantic search
    recs = db.similarity_search(query, k=50)  # Search top 50 similar documents

    # Step 2: Extract ISBNs safely
    isbn_list = []
    for doc in recs:
        text = doc.page_content.strip()
        # Attempt to extract numeric ISBN from the start of the text
        parts = text.split()
        if parts:
            isbn_str = ''.join(filter(str.isdigit, parts[0]))
            if isbn_str.isdigit():
                isbn_list.append(int(isbn_str))

    # Step 3: Filter books by ISBN and return top_k
    recommended_books = books_df[books_df["isbn13"].isin(isbn_list)].head(top_k)

    return recommended_books


In [62]:
# Make sure db_books is your Chroma vector store and books is your DataFrame
query = "A book to teach children about animals"
recommendations = retrieve_semantic_recommendations(query, db_books, books, top_k=10)
print(recommendations)


             isbn13      isbn10                                title  \
368   9780061127762  0061127760    Charlotte's Web Signature Edition   
404   9780064402453  0064402452           Racso and the Rats of NIMH   
406   9780064403870  0064403874  R-T, Margaret, and the Rats of NIMH   
407   9780064404419  0064404412                   The Rainbow People   
416   9780064406925  006440692X                   Winter on the Farm   
429   9780064434980  0064434982                 The Deer in the Wood   
812   9780142302279  0142302279                         Dirty Beasts   
1078  9780241003008  0241003008          The Very Hungry Caterpillar   
1222  9780312319731  0312319738                             Dominion   
1479  9780345434906  0345434900               The Real James Herriot   

                    authors                 categories  \
368             E. B. White           Juvenile Fiction   
404       Jane Leslie Conly           Juvenile Fiction   
406       Jane Leslie Conly      

In [63]:
query = "Stories about friendship and adventure"
recommendations = retrieve_semantic_recommendations(query, db_books, books, top_k=5)
print(recommendations)


            isbn13      isbn10               title             authors  \
123  9780060515225  0060515228      Fragile Things         Neil Gaiman   
215  9780060760441  0060760443   The Reading Group     Elizabeth Noble   
401  9780064400961  0064400964          Betsy-Tacy  Maud Hart Lovelace   
527  9780099498582  0099498588            Dublin 4        Maeve Binchy   
768  9780141187198  0141187190  Seven Gothic Tales        Isak Dinesen   

           categories                                          thumbnail  \
123           Fiction  http://books.google.com/books/content?id=ggLzJ...   
215           Fiction  http://books.google.com/books/content?id=IagWj...   
401  Juvenile Fiction  http://books.google.com/books/content?id=tfUCG...   
527  Domestic fiction  http://books.google.com/books/content?id=yPgE7...   
768           Fiction  http://books.google.com/books/content?id=4uKAn...   

                                           description  published_year  \
123  A mysterious circus